# An Analysis of Deep Neural Network Models for Practical Applications

## Abstract  

自从深度神经网络（DNN）作为计算机视觉领域的一项重要技术出现以来，ImageNet 分类挑战在推进最先进的技术方面发挥了重要作用。在准确率稳步上升的情况下，并没有对胜出模型的资源利用情况进行合适的分析。 在这项工作中，我们对实际应用中的重要指标进行了综合分析：准确率，内存占用，参数，操作次数，推理时间和功耗。主要研究结果如下：（1）功耗与批量和架构无关; （2）准确率和推理时间处于双曲线关系; （3）能量约束是最大可达精度和模型复杂度的上界; （4）操作次数是推断时间的可靠估计。我们相信我们的分析提供了有用的信息，有助于设计高效的 DNN。

## 1 Introduction  

自从 AlexNet作为第一个使用 DNN 的项目 在 2012 ImageNet 竞赛实现了突破以来，为获得更好的性能，一些更复杂的DNN实例被提交到ImageNet 竞赛。  

在ImageNet分类挑战中，最终目标是在多类分类问题框架中获得最高精度，而不管实际的推理时间如何。我们认为这引起了几个问题。首先，参赛者会使用每副验证图像的多个相似实例去训练多个模型，通过模型平均或DNN集成，在付出推理时间或运算量显著提高的代价下，获得报告或论文中描述的足够高的准确率。其次，模型选择受到以下事实的阻碍：不同提交作品使用不同次数的验证图像评估它们的模型，导致不同抽样方式或集成大小下模型准确率与报告中有偏差。第三，目前还没有加快推理时间的动机，这是这些模型在实际应用中的一个关键因素，并且会影响资源利用率，功耗和时间。  

本文旨在比较过去 4 年来提交给 ImageNet 挑战的最先进的 DNN 体系结构，从计算要求和准确性的角度出发。我们将这些体系结构与实际部署中与资源利用率相关的多个度量标准进行比较：准确率，内存占用空间，参数，操作计数，推理时间和功耗。本文的目的是强调这些数字的重要性，这些数字是在实际部署和应用中优化这些网络的重要约束。

## 2 Method  

为了比较不同模型的质量，我们收集并分析了文献报道的准确率,很快发现不同的抽样技术不能直接比较资源利用率。比如，VGG-16 和 GoogleNet 的单次运行 central-crop（top-5验证）误差分别是8.7%和10.07%，表明 VGG-16 性能优于 googleNet；而采用 10-crop 抽样时，两者误差分别是 9.33% 和9.15%，VGG-16 却比 GoogleNet 差了。基于此，我们决定对所有网络统一采用单个 central-crop 抽样方法重新评估 top-1 准确率。  

对于推理时间和内存占用，我们使用 cuDNN-v5 和 CUDA-v8 配置的 Torch7 来进行评估。所有试验使用的都是 JetPack-2.3 英伟达 Jetson TX1 板卡，该板卡内置了一块 64 位 ARM A57CPU 的视觉计算系统、一块 1T-Flop/s 的 256 核英伟达 Maxwell GPU 和 4GB 的 LPDDR4 共享 RAM。使用这种限量级的设备是为了更好地强调网络架构的差异，但是使用K40或Titan X等大多数最新的GPU可获得相似的结果。指令数使用我们开发的开源工具包评估。功耗评估使用的是 Keysight 1146B Hall 电流探头，内置 Keysight MSO-X 2024A 200MHz 数字显波器，采样周期 2s，采样率 50kSa/s，该系统由 Keysight E3645A GPIB数控直流电源供电。

## 3 Results  

在本节中，会报告我们的比较结果。我们分析了以下 DNNs：AlexNet，batch normalised AlexNet，batch normalised NIN，ENet，GoogLeNet，VGG-16 和 VGG-19，ResNet-18，ResNet-34，ResNet-50，ResNet-101 和 ResNet-152，Inception-v3 和 Inception-v4。因为他们是四年中 Imagenet 任务重获得最好结果的网络。  

### 3.1 Accuracy  

图 1 展示了提交给 ImageNet 比赛的各个网络架构的 1-crop 准确率，最左边的是 AlexNet，最右边的是 Inception-v4。最新的 ResNet 和 Inception 架构相比其他架构准确率至少提高 7%。  

图 2 通过同时可视化计算成本和网络参数数量，从另一个角度展示了各网络更详细的准确率值。最明显的是 VGG，尽管它已经被广泛应用于许多应用中，但无论计算需求还是参数数量，都是迄今为止最昂贵。除VGG 的16层和19层的实现外，其他架构形成了一条斜线，到 Inception 和 ResNet 时，这条线开始变平缓。这表明这些模型在该数据集上达到一个拐点。在这个拐点上，计算复杂度的增加开始超过在准确率上获得的好处。  

### 3.2 Inference time  

图 3 展示了各架构在每个图像上的推理时间随 batch 大小的变化情况。从图中可以看出，VGG 处理一张图像所需时间约 0.2秒，这限制了它在 NVIDIA TX1 上的实时应用；令人惊奇的是，AlexNet 随着batch大小从1变化到64，处理速度提高了3倍，这是由它的全连接层的弱优化导致（due to weak optimisation of its fully connected layers）。 

### 3.3 Power  

图 4 展示了功耗随 batch 大小变化情况，可见大多数情况下功耗与 batch 大小无关。结合图 3 和图 4 可见，AlexNet（batch大小为1）和 VGG（batch大小为2）的低功耗与较慢的推理时间有关。  

### 3.4 Memory  

图 5 展示了内存随 batch 大小变化情况，可见起初最大系统内存使用大小是不变的，内存占用随着 batch 大小的增大而增大。这是由于网络模型的初始内存分配以及批处理时的内存需求随着图像数量的增加而成比例的增大。由图 6可见，对规模小于 100 MB 的网络，初始内存分配不会小于 200 MB，而且随后呈现为参数大小的斜率为 1.3 的线性函数。  

### 3.5 Operations  

在神经网络加速器的自定义实现中，运算量对于预估推理时间和硬件电路大小是必要的。图 7 展示了运算量随推理时间和 batch 变化情况，当 batch 大小为 16 时，每个图像的运算量和推理时间之间存在线性关系。因此，在设计网络时可以控制运算量，以使处理速度保持在实时应用或资源有限的应用的可接受范围内。  

### 3.6 Operations and Power  

图 8展示了功耗和运算量的关系，可以发现不同架构之间没有特定的内存占用关系。当资源利用完全时，通常 batch 大小较大，所有网络的额外消耗大致为 11.8 W，标准偏差为 0.7 W，空闲功率为 1.30W。这是资源完全利用时的最大系统功耗。如果功耗是我们要关注的点，那么可以选择满足最低功耗要求的最慢的架构。  

### 3.7 Accuracy and Throughput  

图 9 展示了准确率与每秒推理数量的关系，可见准确率和每秒推理数量之间存在非凡（non-trivial）的线性上限。给定帧速率后，可以实现的最高准确率与帧速率成线性关系。准确率的线性拟合展示了所有架构的准确率与速度之间的关系。此外，给定一个推理时间，可以得出资源充分利用条件下理论上的最高准确率（chosen a specific inference time, one can now come up with the theoretical accuracy upper bound when resources are fully utilised）。当功耗固定时，我们甚至可以进一步得出能耗限定下的最高准确率，而这可以作为需要在嵌入式系统上运行的网络的基本设计因素考虑。  

### 3.8  Parameters utilisation  

DNN 在利用全部学习能力（参数数量/自由度）方面非常低效。Han 在2015 年利用 DNN 的这个缺陷，使用权重剪枝（weights pruning）、量化（quantisation）和变长编码（variable-length symbol encoding）将网络规模减小了50倍。值得注意的是，使用更高效的架构能够产生更紧凑的表征。如图 10 所示，尽管 VGG 比 AlexNet 准确率更高，但其信息密度更差，这意味着在 VGG 架构中引入的自由度并没有带来准确率上的很大提升。此外，图 10 中可以看出，ENet分数最高，仅用了 VGG 1/24 的参数获得了state-of-the-art。  

## 4 Conclusions  

在本文中，我们分析了在精度，内存，参数，操作次数，推理时间和功耗方面提交给 ImageNet 挑战的多个最新的深度神经网络。我们的目标深入理解网络结构，从而可以在实际应用中采用更有效的网络，并优化实际部署中有限的资源，从而我们为 ImageNet 创建了 ENet 或 Efficient-Network。我们表明准确率和推理时间处于双曲线关系：准确度稍微增加需要花费大量的计算时间。我们显示网络模型中的操作数量可以有效地估计推理时间。我们表明，就运算量而言，能量约束将为最大可达到的准确度和模型复杂性设定特定的上限。最后，我们展示 ENet 在参数空间利用率方面是最好的架构，相对于参AlexNet每个参数压缩多达13倍的信息，相对于 VGG-19 则是24倍。

## 参考  

1 [An Analysis of Deep Neural Network Models for Practical Applications](https://arxiv.org/abs/1605.07678)  
2 [DNN模型实际应用分析](https://baijiahao.baidu.com/s?id=1561909582276581&wfr=spider&for=pc)  